In [1]:
import pandas as pd
import datetime as dt

#grab flight plan
flight_plan = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_20220101-20251231.csv')

#query to only use dates where the event is dock and convert it to a list
dock_days_list = flight_plan[flight_plan['event'] == 'Dock']['datedim'].to_list()

#convert list of date strings into a list of datetime objects
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dock_days_list]

#print the days difference between each date
num_days = pd.Series(dates_list).diff().dt.days.iloc[1:].astype(int).tolist()
print("number of daays til next dock\n",num_days)




number of daays til next dock
 [4, 25, 22, 18, 24, 13, 43, 67, 15, 22, 12, 18, 76, 15, 5, 13, 67, 2, 13, 59, 21, 2, 19, 48, 29, 41, 10, 10, 17, 2, 15, 8, 30, 22, 7, 22, 29, 35, 16, 6, 14, 9, 44, 33, 4, 22, 1, 34, 3, 19, 17, 62, 4, 27, 48, 2, 29, 7, 25, 28, 25, 25]


In [2]:
#grab crew members per dock
crewF = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_crew_20220101-20251321.csv')

crew_per_dock = []

for i, date in enumerate(dock_days_list):
    crewNum = crewF[crewF['datedim'] == date]['crew_count'].tolist()
    Amount = sum(crewNum)
    crew_per_dock.append(Amount)

#number of crew members on board during each dock
print("number of crew per docking\n",crew_per_dock)


number of crew per docking
 [7, 7, 10, 11, 11, 7, 7, 7, 10, 11, 7, 7, 7, 7, 7, 11, 7, 11, 11, 7, 7, 7, 11, 10, 7, 7, 11, 11, 7, 7, 11, 7, 10, 9, 7, 11, 7, 7, 11, 7, 7, 10, 10, 11, 7, 7, 7, 7, 7, 11, 7, 10, 11, 11, 7, 7, 11, 7, 10, 7, 7, 11, 7]


In [3]:
#grab rates
rater = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\rates_definition.csv')


#consumable must match a consumable on the rates csv. case sensitive.
consumable = 'Pretreat Tanks'
consumable = 'Air'
consumable = 'Water'


#sum of generated
sum_generated = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'generation')]['rate'].to_list())

#grabs the usage rate of consumable and sums it up
sum_usage = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['rate'].to_list())

#if units contains 'crew'
crew_check = rater[rater['affected_consumable'] == consumable]['units'].tolist()
combine =  '\t'.join(crew_check)

print(crew_check)
if 'Crew' in combine:
    per_crew = True
else:
    per_crew = False

print(per_crew)

#prints
print("Consumable used = ", sum_usage)
print("Consumable generated = ", sum_generated)
#net_consumable = sum_generated - sum_usage
#print("net consumable per day= ", net_consumable)



['Liters/Day', 'Liters/Crew/Day', 'Liters/Crew/Day', 'Liters/Day', 'Liters/Day', 'Liters/Day', 'Liters/Day']
True
Consumable used =  16.84
Consumable generated =  23.549999999999997


In [4]:
import math

#create a list of base rates 
listofrates = [sum_usage] * len(dock_days_list)
consumable_to_send = []
current_sum = 0
res_list = []

if(per_crew): #if percrew true change the list of rates
    for i in range(0, len(listofrates)):
        res_list.append(listofrates[i] * crew_per_dock[i])
else:
    res_list = listofrates

#match length and subtract the amount generated to get proper list of rates   
res_list.pop()
res_list = [x - sum_generated for x in res_list]

intermediate = []
for i in range(0, len(res_list)):
    intermediate.append(res_list[i] * num_days[i])
print (intermediate)

#rounding
for i, value in enumerate(intermediate):
    current_sum+=value
    consumable_to_send.append(math.trunc(current_sum))
    current_sum = current_sum - math.trunc(current_sum)
print(consumable_to_send)
    


[377.32, 2358.25, 3186.7000000000007, 2910.42, 3880.56, 1226.29, 4056.19, 6320.11, 2172.7500000000005, 3557.18, 1131.96, 1697.94, 7169.08, 1414.95, 471.65, 2101.97, 6320.11, 323.38, 2101.97, 5565.47, 1980.93, 188.66, 3072.11, 6952.800000000001, 2735.57, 3867.5299999999997, 1616.9, 1616.9, 1603.61, 188.66, 2425.35, 754.64, 4345.500000000001, 2816.22, 660.31, 3557.18, 2735.57, 3301.5499999999997, 2587.04, 565.98, 1320.62, 1303.65, 6373.4000000000015, 5335.7699999999995, 377.32, 2075.2599999999998, 94.33, 3207.22, 282.99, 3072.11, 1603.61, 8980.7, 646.76, 4365.63, 4527.84, 188.66, 4689.01, 660.31, 3621.2500000000005, 2641.24, 2358.25, 4042.25]
[377, 2358, 3187, 2910, 3881, 1226, 4056, 6320, 2173, 3557, 1132, 1698, 7169, 1415, 472, 2102, 6320, 323, 2102, 5566, 1981, 188, 3072, 6953, 2736, 3867, 1617, 1617, 1604, 188, 2426, 754, 4346, 2816, 660, 3558, 2735, 3302, 2587, 566, 1320, 1304, 6373, 5336, 378, 2075, 94, 3207, 283, 3072, 1604, 8981, 647, 4365, 4528, 189, 4689, 660, 3621, 2641, 2359,

In [5]:
##RUN ALL or this wont work
dock_days_list.pop()

readable = pd.DataFrame(
    {'Docking Days': dock_days_list,
     'Amount of Consumable to Send': consumable_to_send
    })

display(readable)

,Docking Days,Amount of Consumable to Send
0,2/17/2022,377
1,2/21/2022,2358
2,3/18/2022,3187
3,4/9/2022,2910
4,4/27/2022,3881
...,...,...
57,9/3/2025,660
58,9/10/2025,3621
59,10/5/2025,2641
60,11/2/2025,2359
